In [99]:
from __future__ import division, absolute_import
import numpy as np
import time
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from scipy import stats
from scipy.spatial.distance import squareform, pdist
import scipy.cluster.hierarchy as hierarchy
import glob
import math
import re

from cluster_table import sort_clusters_by_attribute

plt.rc('text', usetex = True)
plt.rc('font', family = 'serif')

import os

import json, requests

from notebook import notebookapp
servers = list(notebookapp.list_running_servers())
localhost_name = servers[0]['url']

random.seed(19680801)

<b>class and function definitions</b>

In [152]:
class AMUSE_star:
    def __init__(self, x, y, z, vx, vy, vz):

        #measured data for the variable, kpc and km/s
        self.x = x
        self.y = y
        self.z = z
        
        self.vx = vx
        self.vy = vy
        self.vz = vz

def mode(vals):
    #vals must be a list
    return max(set(vals), key=vals.count)

# returns index of list element closest to a provided value
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

# http://www.mathworks.com/matlabcentral/fileexchange/24693-ellipsoid-fit
# for arbitrary axes
def ellipsoid_fit(X):
    x = X[:, 0]
    y = X[:, 1]
    z = X[:, 2]
    D = np.array([x * x + y * y - 2 * z * z,
                 x * x + z * z - 2 * y * y,
                 2 * x * y,
                 2 * x * z,
                 2 * y * z,
                 2 * x,
                 2 * y,
                 2 * z,
                 1 - 0 * x])
    d2 = np.array(x * x + y * y + z * z).T # rhs for LLSQ
    u = np.linalg.solve(D.dot(D.T), D.dot(d2))
    a = np.array([u[0] + 1 * u[1] - 1])
    b = np.array([u[0] - 2 * u[1] - 1])
    c = np.array([u[1] - 2 * u[0] - 1])
    v = np.concatenate([a, b, c, u[2:]], axis=0).flatten()
    A = np.array([[v[0], v[3], v[4], v[6]],
                  [v[3], v[1], v[5], v[7]],
                  [v[4], v[5], v[2], v[8]],
                  [v[6], v[7], v[8], v[9]]])

    center = np.linalg.solve(- A[:3, :3], v[6:9])

    translation_matrix = np.eye(4)
    translation_matrix[3, :3] = center.T

    R = translation_matrix.dot(A).dot(translation_matrix.T)

    evals, evecs = np.linalg.eig(R[:3, :3] / -R[3, 3])
    evecs = evecs.T

    radii = np.sqrt(1. / np.abs(evals))
    radii *= np.sign(evals)

    return center, evecs, radii

#goes from index to color from the colormap cm
def get_color(ind, N_colors):
    
    cm = plt.get_cmap('jet')
    color_vals = [cm(1.*i/N_colors) for i in range(N_colors)]
    
    return color_vals[ind]

r_sun = 8. #kpc, Vivas & Zinn 2006
r_MW = 15.
n0, qH, nH = 4.5, 0.71, 2.62 #kpc^-3, constants describing halo geometry
#z_sun << r_sun, large error bar so we will assume it's 0

#theoretical distribution
def numdens_RR(R,Z):
    return n0*(r_sun/(R**2 + (Z/qH)**2))**nH

#Gaussian KDE based on distribution of ordered pairs
def density_estimation(m1, m2):
    #memoselyk on stack overflow
    xmin, xmax, ymin, ymax = min(m1), max(m1), min(m2), max(m2)
    X, Y = np.mgrid[xmin:xmax:100j, ymin:ymax:100j]                                                     
    positions = np.vstack([X.ravel(), Y.ravel()])                                                       
    values = np.vstack([m1, m2])                                                                        
    kernel = stats.gaussian_kde(values)                                                                 
    Z = np.reshape(kernel(positions).T, X.shape)
    return X, Y, Z

#checks a list to see if there are nans in it
def no_nans(lst):
    checker = [ 0 if math.isnan(lst[k]) == False else 1 for k in range(len(lst)) ]
    if np.sum(checker) == 0:
        return True
    else:
        return False

'''
Functions needed for RR Lyrae catalog to number density map of clusters pipeline
'''

#from RR_Lyrae variable objects to data frame containing RR Lyrae variable data
def stars_matrix(snapshot, Norbiters):
    
    '''
    returns matrix of all the stars in the sample
    columns: x, y, z, vx, vy, vz
    '''
    
    datadir_AMUSE = '/Users/BrianTCook/Desktop/Thesis/second_project_gcs/Enbid-2.0/AMUSE_data/'
    filename = 'enbid_tree_frame_' + snapshot + '_Norbiters_' + str(Norbiters) + '.ascii'
    X = np.loadtxt(datadir_AMUSE + filename)
        
    return X
    
def condensed_distances_and_linkage(X, method):
    
    '''
    takes phase space coordinate matrix
    computes separation distance, linkage matrix based on proffered method (single, average, and so on)
    '''

    '''
    X_adjusted = np.zeros(X.shape)
    
    for i in range(6):
    
        X_adjusted[:,i] = [ (x - min(X[:,i])) / () for x in X[:,i] ]
    '''
    
    X_spatial = X[:, :3]
    
    print(X_spatial.shape)
    
    cds = pdist(X_spatial, 'euclidean') 
    Z = hierarchy.linkage(cds, method=method)
    
    return cds, Z

def tree_cutter(N_cuts, N_variables, min_height, max_height, Z, N_sizes):
    
    '''
    cuts up tree encoded by linkage matrix N_cut times
    the max_height is the maximum clustering scale, so ~r_MW would be good
    returns N_sizes clustering scales and the relevant information from Z 
    '''
    
    heights = np.logspace(np.log10(max_height), np.log10(min_height), N_cuts) #smallest structure is 100 pc
    
    tree_cuts = [ hierarchy.cut_tree(Z, height=height) for height in heights ] 
    
    groupings_and_counts = [ np.unique(tc,  return_counts=True) for tc in tree_cuts ]
    groupings = [ gc[0] for gc in groupings_and_counts ]
    allocations = [ gc[1] for gc in groupings_and_counts ]
    
    means = [ np.mean(allocation) for allocation in allocations ]
    stdevs = [ np.std(allocation) for allocation in allocations ]
    
    Js = range(4) #show N-1 levels of standard deviation
    Ys = [ [ (means[i] + j * stdevs[i])/N_variables for i in range(N_cuts) ] for j in Js ]
    
    N_groups = [ len(g) for g in groupings ]
    frac_above_3sig = [ 1./N_groups[k] * np.sum([ 1 if alloc >= Ys[3][k]*N_variables else 0 
                        for alloc in allocations[k] ]) for k in range(N_cuts) ]
    
    #way to collect clustering scales, sort by best ones and then pick out local maxima
    best_clustering_heights = sorted(frac_above_3sig, reverse=True)
    bch_indices = []
    ind, flag = 0, 0 #the first one can't be a peak by definition
    
    while flag == 0:
        
        if len(bch_indices) >= N_sizes or ind > N_cuts:
            flag = 1
        
        try:
            bch_value = best_clustering_heights[ind]
            ind_to_try = frac_above_3sig.index(bch_value)
        except:
            flag = 1
            
        try:

            if ind_to_try == 0:
                
                if frac_above_3sig[ind_to_try] > frac_above_3sig[ind_to_try+1]:
                    if ind_to_try not in bch_indices:
                        bch_indices.append(ind_to_try)
                    ind += 1
                else:
                    ind += 1
                
            if ind_to_try == len(frac_above_3sig)-1:
                
                if frac_above_3sig[ind_to_try] > frac_above_3sig[ind_to_try-1]:
                    if ind_to_try not in bch_indices:
                        bch_indices.append(ind_to_try)
                    ind += 1
                else:
                    ind += 1
                
            else:
            
                if frac_above_3sig[ind_to_try] > frac_above_3sig[ind_to_try-1] and frac_above_3sig[ind_to_try] > frac_above_3sig[ind_to_try+1]:
                    if ind_to_try not in bch_indices:
                        bch_indices.append(ind_to_try)
                    ind += 1
                else:
                    ind += 1
                    
        except:
            print('problem with if/else statement')
            flag = 1
    
    from_cut = [ heights, tree_cuts, groupings_and_counts, 
                 groupings, allocations, means, stdevs, bch_indices]
    
    return from_cut

def extract_from_best_cuts(from_cut):
    
    heights = from_cut[0]
    tree_cuts = from_cut[1]
    groupings_and_counts = from_cut[2]
    groupings = from_cut[3]
    allocations = from_cut[4]
    means = from_cut[5]
    stdevs = from_cut[6]
    bch_indices = from_cut[7]
    
    N_sizes_updated = len(bch_indices)
    keep_these_groups = [0]*N_sizes_updated
    point_colors_all = [0]*N_sizes_updated

    mode_color = (0., 0., 0., 0.8)

    #iterates through selected clustering scales, assigns each a variable a color based on cluster identification
    for j in range(N_sizes_updated):

        bch_i = bch_indices[j]

        best_height = heights[bch_i]  
        best_tree_cut = tree_cuts[bch_i]
        best_grouping = groupings[bch_i]
        best_allocation = allocations[bch_i]
        best_mean =  float(means[bch_i])
        best_stdev =  (stdevs[bch_i])

        ktg = [ i for i in range(len(best_allocation)) if best_allocation[i] > (best_mean + 3*best_stdev) ]
        
        point_colors = [ get_color(ktg.index(element), len(ktg)) if element in ktg else mode_color
                              for element in best_tree_cut ] 
        
        keep_these_groups[j] = ktg
        point_colors_all[j] = point_colors

    return point_colors_all
        
def clusters_into_dataframe(N_sizes, point_colors_all, X_total, N_star_min, N_star_max, r_min, r_max):
    
    mode_color = (0., 0., 0., 0.8)
    x_means, y_means, z_means, radii, N_stars, cluster_labels, eccentricities = [], [], [], [], [], [], []

    N_sizes = len(point_colors_all)
    N_variables = len(X_total)
    
    print('N_sizes: ', N_sizes)
    
    #iterates through scales
    for j in range(N_sizes):

        point_colors = point_colors_all[j]

        clustered_colors = [ point_colors[k] for k in range(N_variables) if point_colors[k] != mode_color ]
        color_list = list( set(clustered_colors) )    
        
        x_j, y_j, z_j, rm_j, N_j, label_j, eccentricity_j = [], [], [], [], [], [], []
        
        #each cluster at this scale height
        for cl in list(color_list):

            #kpc
            xs_cl = [ X_total[k,0] for k in range(N_variables) if point_colors[k] == cl ]
            ys_cl = [ X_total[k,1] for k in range(N_variables) if point_colors[k] == cl ] 
            zs_cl = [ X_total[k,2] for k in range(N_variables) if point_colors[k] == cl ] 

            N_cl = len(xs_cl)
            
            X_clust = np.zeros((N_cl, 3))
    
            X_clust[:,0] = xs_cl
            X_clust[:,1] = ys_cl
            X_clust[:,2] = zs_cl
            
            #gets ellipsoidal characteristics of the cluster
            
            try:
                center_clust, evecs_clust, radii_clust = ellipsoid_fit(X_clust)
                min_radius, max_radius = min(radii_clust), max(radii_clust)
               
            except:
                continue
            
            eccentricity = np.sqrt( 1 - (min_radius**2)/(max_radius**2) )
            
            if eccentricity <= 0.2 and max_radius < 0.2: #globular cluster, unlikely to find any

                label_j.append( 'GC' )

            elif eccentricity >= 0.7: #stellar stream

                label_j.append( 'SS' )
                
            else:#if eccentricity > 0.2 and eccentricity < 0.7: #other Sesar-like object

                label_j.append( 'Other' )
                
            x_j.append(np.median(xs_cl))
            y_j.append(np.median(ys_cl))
            z_j.append(np.median(zs_cl))
            rm_j.append(max_radius)
            N_j.append(N_cl)
            eccentricity_j.append(eccentricity)
            
        if len(x_j) != 0:
            
            x_means.append(x_j)
            y_means.append(y_j)
            z_means.append(z_j)
            radii.append(rm_j)
            N_stars.append(N_j)
            cluster_labels.append(label_j)
            eccentricities.append(eccentricity_j)
    
    x_means = [ j for i in x_means for j in i] 
    y_means = [ j for i in y_means for j in i] 
    z_means = [ j for i in z_means for j in i] 
    radii = [ j for i in radii for j in i] 
    N_stars = [ j for i in N_stars for j in i] 
    cluster_labels = [ j for i in cluster_labels for j in i] 
    eccentricities = [ j for i in eccentricities for j in i] 
    
    headers = ['X', 'Y', 'Z', 'Radius', 'Nstars', 'Cluster Tag', 'Eccentricity' ]
    data = [ x_means, y_means, z_means, radii, N_stars, cluster_labels, eccentricities ]
    
    data_dictionary = {}
    
    for i, header in enumerate(headers):
        data_dictionary.update( {header : data[i]} )
    
    df_all = pd.DataFrame(data=data_dictionary)
    
    #matching initial star cluster attributes
    df = df_all[df_all['Nstars'] >= N_star_min]
    df = df[df['Nstars'] <= N_star_max]

    df = df[df['Radius'] >= r_min]
    df = df[df['Radius'] <= r_max]

    #gets rid of redundant clusters, rounds and stores N_stars as an integer
    df = df.drop_duplicates()
    df = df.round(3)
    df['Nstars'] = df['Nstars'].astype(int)
    
    #drop rows with nans in them
    df = df.dropna()
    
    df = df.sort_values(by=['Nstars'])
    df = df.reset_index(drop=True)
    
    print(df.head())
    
    return df, df.to_latex(index=True)           

<b>plotting things</b>

In [153]:
#condensed distance matrix histograms
def sep_dist_distributions(condensed_distances_matrix, condensed_distances_matrix_theory, deltaM):
    
    plt.figure()
    
    plt.hist(condensed_distances_matrix, bins=50, histtype='step', 
             color='g', density=True, label=r'Combined Catalog, $\delta M_{V} = %.02f$'%(deltaM))
        
    plt.hist(condensed_distances_matrix_theory, bins=50, histtype='step',
             color='r', density=True, label=r'From $\rho_{\mathrm{model}}^{RR}(R,Z)$')
    
    plt.legend(loc='best', fontsize=16)
    plt.xlabel('$D(\mathbf{r}_{i}, \mathbf{r}_{j})$ (kpc)', fontsize=20)
    plt.ylabel('Normalized Distribution', fontsize=16)
    plt.gca().tick_params(axis='both', labelsize='large')
    plt.title('Condensed Distance Matrix Elements', fontsize=14)
    plt.tight_layout()
    plt.savefig('distances_distribution_total.pdf')
  
def cluster_population_stat_plot(N_variables, N_cuts, heights, means, stdevs):
    
    Js = range(4)
    Ys = [ [ (means[i] + j * stdevs[i])/N_variables for i in range(N_cuts) ] for j in Js ]

    plt.figure()

    for j in Js:
        if j == 0:
            plt.loglog(heights,Ys[j], 'k', linewidth=0.75, label=r'$\mu$')
        else:
            plt.loglog(heights,Ys[j], linewidth=0.75, label=r'$\mu + %i \sigma$'%(j))

    plt.ylim(1./N_variables,1.)
    plt.legend(loc='best', fontsize = 14)
    #plt.title('%s'%(catalog_name), fontsize=20)
    plt.xlabel(r'$\min\left(L(r,s)\right)$ (kpc)', fontsize=20)
    plt.ylabel(r'Variables Per Cluster (normalized)', fontsize=14)
    plt.gca().tick_params(axis='both', labelsize='x-large')
    plt.tight_layout()
    plt.savefig('means_and_stdevs_total.pdf') #_%s.pdf'%(catalog_name))

def clustering_peak_plot(N_variables, N_cuts, heights, groupings, allocations, means, stdevs, bch_indices):
    
    N_groups = [ len(g) for g in groupings ]

    Js = range(4)
    Ys = [ [ (means[i] + j * stdevs[i])/N_variables for i in range(N_cuts) ] for j in Js ]
    
    frac_above_3sig = [ 1./N_groups[k] * np.sum([ 1 if alloc >= Ys[3][k]*N_variables else 0 
                        for alloc in allocations[k] ]) for k in range(N_cuts)]

    plt.figure()
    plt.semilogx(heights, frac_above_3sig, 'k', linewidth=0.75)
    plt.xlim(min(heights),max(heights))
    #plt.title('%s'%(catalog_name), fontsize=20)
    plt.xlabel(r'$\min\left(L(r,s)\right)$ (kpc)', fontsize=20)
    plt.ylabel(r'$N_{\mathrm{clusters}}$ ($N_{\mathrm{variables}} \geq \mu + 3\sigma$) (normalized)', fontsize=10)

    #need to control for case where sigma > mu, indicating not a good clustering
    best_clustering_heights = sorted(frac_above_3sig, reverse=True)
    
    ci = 0

    N_colors = len(bch_indices)
    
    for bch_i in bch_indices:
        plt.axvline(x=heights[bch_i], linewidth=0.5, 
                    color=get_color(ci, N_colors), label = '%.02f kpc'%(heights[bch_i]))
        ci += 1

    plt.xlim(0, max(heights)+2)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=6)  
    plt.gca().tick_params(axis='both', labelsize='large')
    plt.tight_layout()
    plt.savefig('finding_clustering_peaks_total.pdf') #_%s.pdf'%(catalog_name))

<b>going through several RR Lyrae catalogs</b>

In [174]:
def main():

    print('main has started!')
    t0 = time.time()
    
    snapshot, Norbiters = '00000', 16
    
    X_total = stars_matrix(snapshot, Norbiters)
    N_variables = len(X_total[:,0])

    method = 'average'
    cds, Z = condensed_distances_and_linkage(X_total, method)   
    
    datadir_data = '/Users/BrianTCook/Desktop/Thesis/second_project_gcs/data/'
    cluster_populations = list( np.loadtxt(datadir_data + 'Nstars_in_clusters.txt') ) 
    cluster_radii = list( np.loadtxt(datadir_data + 'ICs/cluster_radii_for_sampling.txt') )
    
    #need to give clusters sorted by an attribute, in our case increasing |r|
    #new_index = indices_dict[old_index]
    indices_dict = sort_clusters_by_attribute('|r|')
    
    cluster_populations_sorted = [ int(cluster_populations[indices_dict[i]]) for i in range(Norbiters) ]
    cluster_radii_sorted = [ cluster_radii[indices_dict[i]] for i in range(Norbiters) ]

    for k, number_of_stars in enumerate(cluster_populations_sorted):
        
        starting_index = int(np.sum( cluster_populations_sorted[:k] ))
        ending_index = starting_index + int(number_of_stars)
        
        print('%i th cluster: (%.03f, %.03f, %.03f) kpc'%(k, np.median(X_total[starting_index:ending_index, 0]),
                                                          np.median(X_total[starting_index:ending_index, 1]),
                                                          np.median(X_total[starting_index:ending_index, 2])))

    N_star_min, N_star_max = 0.9*min(cluster_populations_sorted) , 1.1*max(cluster_populations_sorted)
    r_min, r_max = 0.1*min(cluster_radii_sorted)/1000., 10.*max(cluster_radii_sorted)/1000. #in kpc

    print('r_min, r_max: ', r_min, r_max)
    
    N_sizes, N_cuts, min_height, max_height = 50, 100, 1e-4, 2e-2 #max_height ~ r_MW
    from_cut = tree_cutter(N_cuts, N_variables, min_height, max_height, Z, N_sizes)
    
    heights = from_cut[0]
    tree_cuts = from_cut[1]
    groupings_and_counts = from_cut[2]
    groupings = from_cut[3]
    allocations = from_cut[4]
    means = from_cut[5]
    stdevs = from_cut[6]
    bch_indices = from_cut[7]

    point_colors_all = extract_from_best_cuts(from_cut)
    
    #get clusters
    df, df_as_latex = clusters_into_dataframe(N_sizes, point_colors_all, 
                                              X_total, N_star_min, N_star_max, r_min, r_max)

    df = df.reset_index(drop=True)
    df_as_latex = df.to_latex(index=True) 

    print('have catalog of clusters, time = %.02f minutes'%((time.time()-t0)/60.))
    print('done!')

In [175]:
if __name__ in '__main__':
    
    main()

main has started!
(8761, 3)
0 th cluster: (-0.005, -0.004, -0.018) kpc
1 th cluster: (-0.012, 0.008, -0.012) kpc
2 th cluster: (-0.032, -0.026, 0.023) kpc
3 th cluster: (-0.008, -0.021, 0.043) kpc
4 th cluster: (-0.046, -0.031, -0.010) kpc
5 th cluster: (0.049, -0.027, -0.014) kpc
6 th cluster: (0.040, 0.059, 0.052) kpc
7 th cluster: (0.076, -0.070, -0.084) kpc
8 th cluster: (0.070, 0.073, -0.094) kpc
9 th cluster: (-0.135, -0.023, -0.033) kpc
10 th cluster: (-0.139, 0.016, -0.019) kpc
11 th cluster: (-0.065, 0.064, 0.109) kpc
12 th cluster: (-0.056, 0.131, 0.017) kpc
13 th cluster: (-0.102, 0.030, -0.107) kpc
14 th cluster: (-0.124, 0.062, 0.153) kpc
15 th cluster: (-0.268, 0.027, 0.038) kpc
r_min, r_max:  3.724962826067282e-05 0.0694076334843335
N_sizes:  17


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:306: RuntimeWarning: invalid value encountered in sqrt


       X      Y      Z  Radius  Nstars Cluster Tag  Eccentricity
0 -0.012  0.008 -0.012   0.000     244          SS         0.783
1 -0.056  0.131  0.017   0.000     251          SS         0.913
2  0.076 -0.070 -0.084   0.001     266          SS         0.938
3 -0.139  0.016 -0.019   0.000     272          SS         0.791
4 -0.012  0.008 -0.012   0.000     282          SS         0.741
have catalog of clusters, time = 6.39 minutes
done!
